# Agent Loop Prototype

This notebook demonstrates:
- Initializing the CPG Decision Agent
- Testing the agentic AI loop
- Tool orchestration and automatic tool selection
- Multi-turn conversations with memory
- Using Hugging Face (FREE) or OpenAI models


In [ ]:
# Import agent components
import sys
from pathlib import Path
sys.path.insert(0, str(Path().absolute().parent))

from src.agent.agent_core import CPGDecisionAgent
from src.agent.memory import SessionMemory
from src.genai.llm_interface import LLMInterface
import os

print("✅ Libraries imported successfully!")


In [ ]:
# Initialize LLM (choose one option)

# Option 1: Hugging Face (FREE - Recommended for testing)
print("🤗 Initializing with Hugging Face (FREE)...")
llm = LLMInterface(
    use_huggingface=True,
    huggingface_model="mistralai/Mistral-7B-Instruct-v0.2"
)

# Option 2: OpenAI (Uncomment to use OpenAI instead)
# print("🔵 Initializing with OpenAI...")
# llm = LLMInterface(model="gpt-4")  # or "gpt-3.5-turbo" for cheaper option

# Initialize memory and agent
memory = SessionMemory()
data_path = '../data/cpg_sales_data.parquet'

if os.path.exists(data_path):
    agent = CPGDecisionAgent(
        llm=llm,
        memory=memory,
        data_path=data_path
    )
    print(f"✅ Agent initialized with data from {data_path}")
else:
    agent = CPGDecisionAgent(
        llm=llm,
        memory=memory
    )
    print("⚠️  Agent initialized without data. Load data manually or provide data_path.")


In [ ]:
## Test Agent Loop - Basic Question


# Test agent with a basic question
question = "What are the sales trends for the last quarter?"

print(f"❓ Question: {question}\n")
print("🤖 Agent processing...\n")

try:
    result = agent.run(question, generate_memo=False)
    
    print("=" * 60)
    print("📋 Agent Response:")
    print("=" * 60)
    print(result.get('response', 'No response generated'))
    
    if result.get('tool_calls'):
        print("\n" + "=" * 60)
        print("🔧 Tools Used:")
        print("=" * 60)
        for tool_call in result['tool_calls']:
            print(f"   - {tool_call.get('tool_name', 'Unknown')}")
    
    if result.get('analysis_results'):
        print("\n" + "=" * 60)
        print("📊 Analysis Results:")
        print("=" * 60)
        print(result['analysis_results'])
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


## Test Tool Orchestration


In [ ]:
# Test agent with a scenario question (should trigger simulation tools)
question = "What would happen if we increase price by 10% for product 101?"

print(f"❓ Question: {question}\n")
print("🤖 Agent processing...\n")

try:
    result = agent.run(question, generate_memo=False)
    
    print("=" * 60)
    print("📋 Agent Response:")
    print("=" * 60)
    print(result.get('response', 'No response generated'))
    
    if result.get('tool_calls'):
        print("\n" + "=" * 60)
        print("🔧 Tools Used:")
        print("=" * 60)
        for tool_call in result['tool_calls']:
            print(f"   - {tool_call.get('tool_name', 'Unknown')}")
            if 'params' in tool_call:
                print(f"     Parameters: {tool_call['params']}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


## Test Memory and Multi-turn Conversations


In [ ]:
# Test multi-turn conversation with memory
print("💬 Testing multi-turn conversation...\n")

questions = [
    "What are the sales trends?",
    "Compare that with store performance",
    "What anomalies do you see?"
]

for i, question in enumerate(questions, 1):
    print(f"\n{'='*60}")
    print(f"Turn {i}: {question}")
    print('='*60)
    
    try:
        result = agent.run(question, generate_memo=False)
        print(f"\n🤖 Response: {result.get('response', 'No response')[:200]}...")
        
        # Show memory context
        if hasattr(agent.memory, 'get_recent_history'):
            history = agent.memory.get_recent_history(n=3)
            print(f"\n💭 Memory: {len(history)} recent messages")
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n✅ Multi-turn conversation test complete!")


## Check Memory and Context


In [ ]:
# Check session memory
print("📚 Session Memory Overview:")
print("=" * 60)

if hasattr(agent.memory, 'get_recent_history'):
    history = agent.memory.get_recent_history(n=10)
    print(f"Total messages in memory: {len(history)}")
    print("\nRecent conversation:")
    for msg in history[-5:]:  # Show last 5 messages
        role = msg.get('role', 'unknown')
        content = msg.get('content', '')[:100]
        print(f"  [{role}]: {content}...")

if hasattr(agent.memory, 'get_context'):
    context = agent.memory.get_context()
    print(f"\n📋 Context variables: {len(context)}")
    for key, value in list(context.items())[:5]:
        print(f"   {key}: {value}")

print("\n✅ Memory check complete!")
